In [ ]:
#Classificador Gaussiano
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

size = 0.8
reps = 1
λ = 0.5

data = np.loadtxt('../0b. DADOS/EMGsDataset.csv', delimiter=',')

C1, C2, C3, C4, C5 = 1, 2, 3, 4, 5
results = []
print(data)

In [ ]:
X = np.concatenate((
  data[:2, data[2, :] == C1],
  data[:2, data[2, :] == C2],
  data[:2, data[2, :] == C3],
  data[:2, data[2, :] == C4],
  data[:2, data[2, :] == C5]
), axis=1)

p, N = X.shape

print(X)

Y = np.concatenate((
  np.tile(C1 - 1, (10000, 1)),
  np.tile(C2 - 1, (10000, 1)),
  np.tile(C3 - 1, (10000, 1)),
  np.tile(C4 - 1, (10000, 1)),
  np.tile(C5 - 1, (10000, 1))
)).reshape(1, N)

print(Y)

In [ ]:
results = []

for _ in range(reps):
  indices = np.arange(N)
  np.random.shuffle(indices)
  
  ind_treino = indices[:int(N*size)]
  ind_teste = indices[int(N*size):]
  
  X_treino, X_teste = X[:, ind_treino], X[:, ind_teste]
  Y_treino, Y_teste = Y[:, ind_treino], Y[:, ind_teste]
  
  X_group = []
  Cov_group = []
  
  for i in range(5):
    Xni = X_treino[:, Y_treino[0, :] == i]
    X_group.append(Xni)      
    Cov_group.append(np.cov(Xni))
  
  Means_group = []
  for i in X_group:
    Means_group.append(np.mean(i, axis=1).reshape(p, 1))
    
  Agr_matrix = np.zeros((p,p))
  for i in range(5):
    ni = X_group[i].shape[1]
    Agr_matrix += ni/N * Cov_group[i]
    
  Friedman_group = []
  for i in range(5):
    ni = X_group[i].shape[1]
    dividendo = (1 - λ)*(ni * Cov_group[i])+(λ * N * Agr_matrix)
    divisor = (1 - λ) * ni + λ * N
    Friedman_group.append(dividendo/divisor)

  inv_group = []    
  dets_group=[]
  for i in range(5):
    inv_group.append(np.linalg.inv(Friedman_group[i]))
    dets_group.append(np.linalg.det(Friedman_group[i]))
    
  def discriminante(xn, mi, inv, det):
    t1 = -0.5 * np.log(det)
    t2 = -0.5 * (xn - mi).T @ inv @ (xn - mi)
    return t1 + t2

  Y_pred = []
    
  for i in range(X_teste.shape[1]):
    discriminantes_matrix = []
    X_novo = X_teste[:, i].reshape(2, 1)
    for n in range(5):
      discriminantes_matrix.append(discriminante(X_novo, Means_group[n], inv_group[n], dets_group[n]))
    Y_pred.append(np.argmax(discriminantes_matrix))
    
    
  Y_pred = np.array(Y_pred)
  Y_teste = Y_teste[0]
  results.append(np.mean(Y_pred  == Y_teste))

In [ ]:
print("Média da Acurácia: ", np.mean(results))
print("Desvio Padrão: ", np.std(results))
print("Maior: ", np.max(results))
print("Menor: ", np.min(results))